In [17]:
!pip install prophet pmdarima statsmodels numpy pandas scikit-learn --quiet


In [9]:
import pandas as pd

cleaned_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/cleaned_data.parquet"

df = pd.read_parquet(cleaned_path)
print(df.shape)
print(df.columns)
df.head()

(554719, 22)
Index(['Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID',
       'date_received', 'year', 'month_num', 'day', 'clean_text'],
      dtype='object')


,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,...,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,date_received,year,month_num,day,clean_text
0,DEBT COLLECTION,Credit card debt,TOOK OR THREATENED TO TAKE NEGATIVE OR LEGAL A...,SEIZED OR ATTEMPTED TO SEIZE YOUR PROPERTY,Military star card has violated my consumer ri...,Company has responded to the consumer and the ...,Army and Air Force Exchange Service,TX,761XX,Servicemember,...,06/27/21,Closed with explanation,No,N/A,4496008,2021-06-27,2021,6,27,military star card has violated my consumer ri...
1,"MONEY TRANSFER, VIRTUAL CURRENCY, OR MONEY SER...",Domestic (US) money transfer,FRAUD OR SCAM,NO_SUB_ISSUE,"On XX/XX/year>, XXXX charged {$20.00} for a X...",None,"Paypal Holdings, Inc",CT,064XX,None,...,09/10/25,Closed with non-monetary relief,Yes,N/A,15812229,2025-09-09,2025,9,9,on xx xx year charged for a order using a clos...
2,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,Credit reporting,INCORRECT INFORMATION ON YOUR REPORT,INFORMATION BELONGS TO SOMEONE ELSE,"Dear CFPB, I am sending to dispute fraudulent ...",Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",FL,33578,None,...,08/04/25,Closed with explanation,Yes,N/A,15013629,2025-08-04,2025,8,4,dear cfpb i am sending to dispute fraudulent i...
3,CHECKING OR SAVINGS ACCOUNT,Checking account,PROBLEM CAUSED BY YOUR FUNDS BEING LOW,OVERDRAFTS AND OVERDRAFT FEES,I am writing to file a complaint against navy ...,Company believes it acted appropriately as aut...,NAVY FEDERAL CREDIT UNION,NY,14215,None,...,01/19/25,Closed with explanation,Yes,N/A,11623116,2025-01-19,2025,1,19,i am writing to file a complaint against vy fe...
4,CREDIT REPORTING OR OTHER PERSONAL CONSUMER RE...,Credit reporting,IMPROPER USE OF YOUR REPORT,REPORTING COMPANY USED YOUR REPORT IMPROPERLY,These companies are in clear violation of the ...,None,"EQUIFAX, INC.",NC,28278,None,...,04/10/24,Closed with explanation,Yes,N/A,8740050,2024-04-10,2024,4,10,these companies are in clear violation of the ...


In [10]:
import pandas as pd

# Paths
cleaned_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/cleaned_data.parquet"
monthly_save_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated.csv"

# Load cleaned data
df = pd.read_parquet(cleaned_path)

# Ensure date column is datetime
df["date_received"] = pd.to_datetime(df["date_received"], errors="coerce")

# Extract year & month (if not present)
df["year"] = df["date_received"].dt.year
df["month_num"] = df["date_received"].dt.month

# Aggregate monthly complaint counts
monthly_df = (
    df.groupby(["Product", "State", "year", "month_num"])
      .size()
      .reset_index(name="complaint_count")
)

# Create a proper date column for Prophet
monthly_df["date"] = pd.to_datetime(
    monthly_df["year"].astype(str) + "-" + monthly_df["month_num"].astype(str) + "-01"
)

# Sort
monthly_df = monthly_df.sort_values(["Product", "State", "date"])

import pandas as pd

monthly_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated.csv"

monthly_df = pd.read_csv(monthly_path)

# Valid US state codes (51 including DC)
valid_states = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA",
    "HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
    "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ",
    "NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
    "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY","DC"
]

filtered_df = monthly_df[monthly_df["State"].isin(valid_states)].copy()

print("Original rows:", len(monthly_df))
print("Filtered rows:", len(filtered_df))
print("Removed rows:", len(monthly_df) - len(filtered_df))

filtered_df.head()


# Save the result
monthly_df.to_csv(monthly_save_path, index=False)

print("Monthly dataset saved at:", monthly_save_path)
monthly_df.head()


Original rows: 21574
Filtered rows: 21193
Removed rows: 381
Monthly dataset saved at: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated.csv


,Product,State,year,month_num,complaint_count,date
0,CHECKING OR SAVINGS ACCOUNT,AA,2023,4,1,2023-04-01
1,CHECKING OR SAVINGS ACCOUNT,AE,2021,8,1,2021-08-01
2,CHECKING OR SAVINGS ACCOUNT,AE,2022,12,1,2022-12-01
3,CHECKING OR SAVINGS ACCOUNT,AE,2023,4,1,2023-04-01
4,CHECKING OR SAVINGS ACCOUNT,AE,2023,7,1,2023-07-01


In [11]:
import pandas as pd
import os

monthly_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated.csv"
out_valid_pairs = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/valid_pairs_24m.csv"

df = pd.read_csv(monthly_path, parse_dates=["date"])
print("Loaded monthly aggregated:", df.shape)

# count months per pair
pair_counts = (
    df.groupby(["Product", "State"])["date"]
      .nunique()   # unique months
      .reset_index(name="months_available")
      .sort_values(["months_available", "Product", "State"], ascending=[False, True, True])
)

# threshold
MIN_MONTHS = 24
valid_pairs = pair_counts[pair_counts["months_available"] >= MIN_MONTHS].reset_index(drop=True)

print("Total pairs (all):", pair_counts.shape[0])
print(f"Pairs with >= {MIN_MONTHS} months:", len(valid_pairs))

# save for later
os.makedirs(os.path.dirname(out_valid_pairs), exist_ok=True)
valid_pairs.to_csv(out_valid_pairs, index=False)
print("Saved valid pairs to:", out_valid_pairs)

# show top 10 to paste here
valid_pairs.head(10)


Loaded monthly aggregated: (21574, 6)
Total pairs (all): 779
Pairs with >= 24 months: 447
Saved valid pairs to: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/valid_pairs_24m.csv


,Product,State,months_available
0,CHECKING OR SAVINGS ACCOUNT,AZ,60
1,CHECKING OR SAVINGS ACCOUNT,CA,60
2,CHECKING OR SAVINGS ACCOUNT,CT,60
3,CHECKING OR SAVINGS ACCOUNT,FL,60
4,CHECKING OR SAVINGS ACCOUNT,GA,60
5,CHECKING OR SAVINGS ACCOUNT,IL,60
6,CHECKING OR SAVINGS ACCOUNT,MI,60
7,CHECKING OR SAVINGS ACCOUNT,NC,60
8,CHECKING OR SAVINGS ACCOUNT,NJ,60
9,CHECKING OR SAVINGS ACCOUNT,NV,60


In [12]:
import pandas as pd

monthly_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated.csv"
df = pd.read_csv(monthly_path, parse_dates=["date"])

# PICK ONE PAIR — same as before
prod = "CHECKING OR SAVINGS ACCOUNT"
state = "CA"

pair_df = df[(df["Product"] == prod) & (df["State"] == state)].copy()

pair_df = pair_df.sort_values("date").reset_index(drop=True)

print("Shape:", pair_df.shape)
print(pair_df)


Shape: (60, 6)
                        Product State  year  month_num  complaint_count  \
0   CHECKING OR SAVINGS ACCOUNT    CA  2020         11               30   
1   CHECKING OR SAVINGS ACCOUNT    CA  2020         12               18   
2   CHECKING OR SAVINGS ACCOUNT    CA  2021          1               39   
3   CHECKING OR SAVINGS ACCOUNT    CA  2021          2               24   
4   CHECKING OR SAVINGS ACCOUNT    CA  2021          3               29   
5   CHECKING OR SAVINGS ACCOUNT    CA  2021          4               42   
6   CHECKING OR SAVINGS ACCOUNT    CA  2021          5               34   
7   CHECKING OR SAVINGS ACCOUNT    CA  2021          6               26   
8   CHECKING OR SAVINGS ACCOUNT    CA  2021          7               24   
9   CHECKING OR SAVINGS ACCOUNT    CA  2021          8               35   
10  CHECKING OR SAVINGS ACCOUNT    CA  2021          9               23   
11  CHECKING OR SAVINGS ACCOUNT    CA  2021         10               35   
12  CHECKI

In [13]:
import pandas as pd

# pair_df must be the CA–CHECKING OR SAVINGS ACCOUNT dataframe of 60 rows

pair_df = pair_df.sort_values("date").reset_index(drop=True)

# Step 1: Locate the spike
spike_mask = pair_df['date'] == "2025-01-01"

print("Old spike value:", pair_df.loc[spike_mask, "complaint_count"].values)

# Step 2: Compute rolling median from last 12 months BEFORE the spike
rolling_median = (
    pair_df.loc[pair_df.index < pair_df[spike_mask].index[0], "complaint_count"]
    .rolling(12, min_periods=1)
    .median()
    .iloc[-1]
)

print("Replacement value:", rolling_median)

# Step 3: Replace the spike
pair_df.loc[spike_mask, "complaint_count"] = rolling_median

print("\n=== CLEANED TAIL ===")
print(pair_df.tail(15))


Old spike value: [256]
Replacement value: 74.5

=== CLEANED TAIL ===
                        Product State  year  month_num  complaint_count  \
45  CHECKING OR SAVINGS ACCOUNT    CA  2024          8             79.0   
46  CHECKING OR SAVINGS ACCOUNT    CA  2024          9             85.0   
47  CHECKING OR SAVINGS ACCOUNT    CA  2024         10             75.0   
48  CHECKING OR SAVINGS ACCOUNT    CA  2024         11             69.0   
49  CHECKING OR SAVINGS ACCOUNT    CA  2024         12             88.0   
50  CHECKING OR SAVINGS ACCOUNT    CA  2025          1             74.5   
51  CHECKING OR SAVINGS ACCOUNT    CA  2025          2             92.0   
52  CHECKING OR SAVINGS ACCOUNT    CA  2025          3             70.0   
53  CHECKING OR SAVINGS ACCOUNT    CA  2025          4             71.0   
54  CHECKING OR SAVINGS ACCOUNT    CA  2025          5             83.0   
55  CHECKING OR SAVINGS ACCOUNT    CA  2025          6             72.0   
56  CHECKING OR SAVINGS ACCOUNT

In [1]:
import pandas as pd

monthly_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated.csv"
df = pd.read_csv(monthly_path, parse_dates=["date"])

print("Loaded monthly aggregated:", df.shape)
df.head()


Loaded monthly aggregated: (21574, 6)


,Product,State,year,month_num,complaint_count,date
0,CHECKING OR SAVINGS ACCOUNT,AA,2023,4,1,2023-04-01
1,CHECKING OR SAVINGS ACCOUNT,AE,2021,8,1,2021-08-01
2,CHECKING OR SAVINGS ACCOUNT,AE,2022,12,1,2022-12-01
3,CHECKING OR SAVINGS ACCOUNT,AE,2023,4,1,2023-04-01
4,CHECKING OR SAVINGS ACCOUNT,AE,2023,7,1,2023-07-01


In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error

# =============================
# PICK PRODUCT & STATE HERE
# =============================
prod = "CHECKING OR SAVINGS ACCOUNT"
state = "CA"

pair_df = df[(df["Product"] == prod) & (df["State"] == state)].copy()
pair_df = pair_df.sort_values("date").reset_index(drop=True)

print("Time series length:", len(pair_df))
display(pair_df.head(10))

# -------- OPTIONAL: Spike Correction for CA (example) --------
if prod == "CHECKING OR SAVINGS ACCOUNT" and state == "CA":
    spike_mask = pair_df['date'] == "2025-01-01"
    if spike_mask.sum() == 1:
        rolling_median = (
            pair_df.loc[pair_df.index < pair_df[spike_mask].index[0], "complaint_count"]
            .rolling(12, min_periods=1)
            .median()
            .iloc[-1]
        )
        pair_df.loc[spike_mask, "complaint_count"] = rolling_median
        print("Spike corrected using rolling median:", rolling_median)

# ===== Prepare Prophet format =====
pair_df['ds'] = pair_df['date']
pair_df['y'] = pair_df['complaint_count'].astype(float)

# ===== Split train/test =====
H = 6  # 6-month holdout
train = pair_df.iloc[:-H].copy()
test = pair_df.iloc[-H:].copy()

# ===== Metric function =====
def metrics(true, pred):
    mse = mean_squared_error(true, pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(true, pred)
    mape = np.mean(np.abs((true - pred) / np.clip(true, 1e-8, None)))
    return {"mse": mse, "rmse": rmse, "mae": mae, "mape": mape}

# =====================
#     PROPHET MODEL
# =====================
m_prophet = Prophet(yearly_seasonality=True)
m_prophet.fit(train[['ds','y']])

future_train = m_prophet.make_future_dataframe(periods=H, freq="MS")
fc_prophet = m_prophet.predict(future_train)

pred_prophet = fc_prophet[['ds','yhat']].tail(H).reset_index(drop=True)
true_test = test[['ds','y']].reset_index(drop=True)

prophet_metrics = metrics(true_test['y'], pred_prophet['yhat'])

# Train on full data for forward forecast
m_prophet_full = Prophet(yearly_seasonality=True)
m_prophet_full.fit(pair_df[['ds','y']])
future_full = m_prophet_full.make_future_dataframe(periods=12, freq="MS")
fc_prophet_full = m_prophet_full.predict(future_full)

# =====================
#     SARIMA MODEL
# =====================
sarima_metrics = None
sarima_forecast_full = None

try:
    import pmdarima as pm
    arima = pm.auto_arima(train['y'], seasonal=True, m=12,
                          error_action='ignore', suppress_warnings=True)
    pred_sarima = arima.predict(n_periods=H)
    sarima_metrics = metrics(true_test['y'], pred_sarima)

    # full forecast
    arima.fit(pair_df['y'])
    sarima_forecast_full = arima.predict(n_periods=12)

except Exception:
    pass

# =====================
#   PRINT RESULTS
# =====================
print("\n=== HOLDOUT METRICS ===")
print("Prophet:", {k: round(v,4) for k,v in prophet_metrics.items()})
print("SARIMA:", {k: round(v,4) for k,v in sarima_metrics.items()} if sarima_metrics else "FAILED")

print("\n=== HOLDOUT COMPARISON (ACTUAL vs PREDICTED) ===")
for i in range(H):
    print(
        f"{true_test.loc[i,'ds'].strftime('%Y-%m-%d')}  |  "
        f"Actual: {int(true_test.loc[i,'y'])}  |  "
        f"Prophet: {round(pred_prophet.loc[i,'yhat'],2)}  |  "
        f"SARIMA: {round(float(pred_sarima.iloc[i]),2) if sarima_metrics else 'NA'}" # Changed pred_sarima[i] to pred_sarima.iloc[i]
    )

print("\n=== PROPHET FORECAST NEXT 12 MONTHS (TAIL) ===")
print(fc_prophet_full[['ds','yhat','yhat_lower','yhat_upper']].tail(6).to_string(index=False))

if sarima_forecast_full is not None:
    print("\n=== SARIMA FORECAST NEXT 12 MONTHS (TAIL) ===")
    future_dates = pd.date_range(pair_df['ds'].max() + pd.DateOffset(months=1), periods=12, freq="MS")
    sarima_full_df = pd.DataFrame({"ds": future_dates, "yhat": sarima_forecast_full})
    print(sarima_full_df.tail(6).to_string(index=False))

# =====================
#     MODEL CHOICE
# =====================
print("\n=== MODEL RECOMMENDATION ===")
if sarima_metrics is None:
    print("Use PROPHET (SARIMA failed).")
elif prophet_metrics['rmse'] <= sarima_metrics['rmse']:
    print("Use PROPHET (lower RMSE).")
else:
    print("Use SARIMA (lower RMSE).")

Time series length: 60


,Product,State,year,month_num,complaint_count,date
0,CHECKING OR SAVINGS ACCOUNT,CA,2020,11,30,2020-11-01
1,CHECKING OR SAVINGS ACCOUNT,CA,2020,12,18,2020-12-01
2,CHECKING OR SAVINGS ACCOUNT,CA,2021,1,39,2021-01-01
3,CHECKING OR SAVINGS ACCOUNT,CA,2021,2,24,2021-02-01
4,CHECKING OR SAVINGS ACCOUNT,CA,2021,3,29,2021-03-01
5,CHECKING OR SAVINGS ACCOUNT,CA,2021,4,42,2021-04-01
6,CHECKING OR SAVINGS ACCOUNT,CA,2021,5,34,2021-05-01
7,CHECKING OR SAVINGS ACCOUNT,CA,2021,6,26,2021-06-01
8,CHECKING OR SAVINGS ACCOUNT,CA,2021,7,24,2021-07-01
9,CHECKING OR SAVINGS ACCOUNT,CA,2021,8,35,2021-08-01


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Spike corrected using rolling median: 74.5


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



=== HOLDOUT METRICS ===
Prophet: {'mse': 1116.4993, 'rmse': np.float64(33.4141), 'mae': 26.6552, 'mape': np.float64(0.6689)}
SARIMA: {'mse': 713.7321, 'rmse': np.float64(26.7158), 'mae': 20.4743, 'mape': nan}

=== HOLDOUT COMPARISON (ACTUAL vs PREDICTED) ===
2025-05-01  |  Actual: 83  |  Prophet: 83.19  |  SARIMA: 74.92
2025-06-01  |  Actual: 72  |  Prophet: 85.05  |  SARIMA: 74.92
2025-07-01  |  Actual: 107  |  Prophet: 82.57  |  SARIMA: 74.92
2025-08-01  |  Actual: 71  |  Prophet: 90.05  |  SARIMA: 74.92
2025-09-01  |  Actual: 49  |  Prophet: 89.59  |  SARIMA: 74.92
2025-10-01  |  Actual: 25  |  Prophet: 87.62  |  SARIMA: 74.92

=== PROPHET FORECAST NEXT 12 MONTHS (TAIL) ===
        ds      yhat  yhat_lower  yhat_upper
2026-05-01 88.258267   69.449554  109.939725
2026-06-01 90.140409   70.159118  110.121726
2026-07-01 92.882338   73.645870  113.033156
2026-08-01 97.765600   76.994690  117.185091
2026-09-01 95.668560   75.404539  116.201431
2026-10-01 85.110265   65.956365  103.43008

In [17]:
# Full, restart-safe batch forecasting pipeline
# Paste and run as one cell.
# IMPORTANT PATHS (already used in your notebook):
MONTHLY_PATH = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated.csv"
VALID_PAIRS_PATH = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/valid_pairs_24m.csv"
OUTPUT_FOLDER = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/forecasts_batch/"
METRICS_OUT = OUTPUT_FOLDER + "metrics_all_pairs.csv"
FAILURES_OUT = OUTPUT_FOLDER + "failures.csv"
PROPHET_FORECASTS_FOLDER = OUTPUT_FOLDER + "prophet_forecasts/"
SARIMA_FORECASTS_FOLDER = OUTPUT_FOLDER + "sarima_forecasts/"

# CONFIG
HOLDOUT_MONTHS = 6     # evaluate on last 6 months
FORECAST_MONTHS = 12   # final forecast horizon
MIN_MONTHS = 24
RUN_LIMIT = None       # set to e.g. 10 for quick test; None = all pairs
REPLACE_SPIKE_METHOD = "rolling_median_12"  # simple spike fix

# 1) Robustly install required libs (run only once per session)
import sys, subprocess
print("Ensuring prophet and dependencies are installed...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "prophet", "pystan", "cmdstanpy"], check=False)
subprocess.run([sys.executable, "-m", "pip", "install", "prophet", "pmdarima", "statsmodels", "numpy", "pandas", "scikit-learn", "--quiet"], check=False)

# 2) Imports
import os, warnings, math, traceback
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Prophet import
from prophet import Prophet

# Try pmdarima
try:
    import pmdarima as pm
    PM_AVAILABLE = True
except Exception:
    PM_AVAILABLE = False

# Try statsmodels
try:
    import statsmodels.api as sm
    SM_AVAILABLE = True
except Exception:
    SM_AVAILABLE = False

# 3) Ensure output folders exist
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(PROPHET_FORECASTS_FOLDER, exist_ok=True)
os.makedirs(SARIMA_FORECASTS_FOLDER, exist_ok=True)

# 4) Utility helpers
def metrics(true, pred):
    true = np.array(true, dtype=float)
    pred = np.array(pred, dtype=float)
    mse = mean_squared_error(true, pred)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(true, pred)
    # mapes: avoid division by zero; where true==0 use NaN
    with np.errstate(divide='ignore', invalid='ignore'):
        mape = np.mean(np.abs((true - pred) / np.where(true==0, np.nan, true)))
    return {"mse": float(mse), "rmse": float(rmse), "mae": float(mae), "mape": float(np.nan if np.isnan(mape) else mape)}

def safe_filename(s):
    return str(s).replace(" ", "_").replace("/", "_").replace("\\","_")

# spike replace: replace any value > 5 * rolling median of previous 12 months with that rolling median
def fix_spike(series_y, series_dates, spike_date=None):
    df = pd.DataFrame({"ds": pd.to_datetime(series_dates), "y": np.array(series_y, dtype=float)})
    df = df.sort_values("ds").reset_index(drop=True)
    if spike_date is None:
        return df
    mask = df['ds'] == pd.to_datetime(spike_date)
    if mask.sum() == 0:
        return df
    idx = df.index[mask][0]
    if idx == 0:
        return df
    before = df.loc[:idx-1, "y"]
    if len(before) == 0:
        return df
    rolling_med = before.rolling(window=12, min_periods=1).median().iloc[-1]
    if pd.isna(rolling_med):
        return df
    old = df.at[idx, "y"]
    # consider it a spike if older value > 4 * rolling_med and also >= median*2 (simple rule)
    if (old > 4 * rolling_med) and (old >= 2 * np.nanmedian(before)):
        df.at[idx, "y"] = rolling_med
    return df

# 5) Load inputs
print("Loading data...")
monthly_df = pd.read_csv(MONTHLY_PATH, parse_dates=["date"])
pairs_df = pd.read_csv(VALID_PAIRS_PATH)
print("Monthly rows:", monthly_df.shape)
print("Valid pairs:", pairs_df.shape)

# join to ensure canonical columns
monthly_df = monthly_df.rename(columns={"date":"date", "complaint_count":"complaint_count"})
monthly_df['date'] = pd.to_datetime(monthly_df['date'])

# 6) Which pairs to run
pairs = pairs_df[["Product","State"]].drop_duplicates().reset_index(drop=True)
total_pairs = len(pairs)
if RUN_LIMIT is not None:
    pairs = pairs.head(RUN_LIMIT)
print(f"Preparing to run {len(pairs)} pairs (of {total_pairs} available)")

# 7) Load previously saved metrics to skip processed pairs (restart-safe)
if os.path.exists(METRICS_OUT):
    metrics_master = pd.read_csv(METRICS_OUT)
else:
    metrics_master = pd.DataFrame(columns=["Product","State","n_months",
                                           "prophet_mse","prophet_rmse","prophet_mae","prophet_mape",
                                           "sarima_mse","sarima_rmse","sarima_mae","sarima_mape",
                                           "chosen_model","notes"])
# failures
if os.path.exists(FAILURES_OUT):
    failures = pd.read_csv(FAILURES_OUT)
else:
    failures = pd.DataFrame(columns=["Product","State","error","traceback"])

# 8) Main loop
count = 0
for i, r in pairs.iterrows():
    prod = r["Product"]
    st = r["State"]
    key = (prod, st)
    # skip if already done
    if ((metrics_master['Product']==prod) & (metrics_master['State']==st)).any():
        print(f"Skipping {prod} — {st} (already processed)")
        continue

    count += 1
    print(f"\n[{count}] Processing pair: {prod} — {st}")

    try:
        pair_df = monthly_df[(monthly_df["Product"]==prod) & (monthly_df["State"]==st)].copy()
        pair_df = pair_df.sort_values("date").reset_index(drop=True)
        n_months = pair_df['date'].nunique()
        if n_months < MIN_MONTHS:
            note = f"SKIP: only {n_months} months"
            print(note)
            row_to_add = {
                "Product": prod, "State": st, "n_months": n_months,
                "prophet_mse": np.nan, "prophet_rmse": np.nan, "prophet_mae": np.nan, "prophet_mape": np.nan,
                "sarima_mse": np.nan, "sarima_rmse": np.nan, "sarima_mae": np.nan, "sarima_mape": np.nan,
                "chosen_model": "SKIPPED", "notes": note
            }
            metrics_master = pd.concat([metrics_master, pd.DataFrame([row_to_add])], ignore_index=True)
            metrics_master.to_csv(METRICS_OUT, index=False)
            continue

        # Prepare series
        s_dates = pair_df['date'].values
        s_y = pair_df['complaint_count'].astype(float).values

        # Optional spike correction: look for obvious single-month spike equal to max and > 4*median previous 12 months
        # If spike found at last 12 months, we fix it (you used 2025-01-01 earlier — generic rule here)
        # We'll try to find any month where value > 4 * median(prev12) and replace.
        df_temp = pd.DataFrame({"ds": pd.to_datetime(s_dates), "y": s_y}).sort_values("ds").reset_index(drop=True)
        for idx in range(len(df_temp)):
            if idx == 0: continue
            prev12 = df_temp.loc[max(0, idx-12):idx-1, "y"]
            if len(prev12) >= 3:
                med = prev12.median()
                if med > 0 and df_temp.at[idx,"y"] > 4*med and df_temp.at[idx,"y"] > med*2:
                    print("Spike detected at", df_temp.at[idx,"ds"].strftime("%Y-%m-%d"), "old:", df_temp.at[idx,"y"], "->", med)
                    df_temp.at[idx,"y"] = med

        # Now final series with ds/y columns
        df_series = df_temp.copy()

        # train/test split
        if len(df_series) <= HOLDOUT_MONTHS + 6:
            # safety
            raise RuntimeError(f"Not enough points for holdout H={HOLDOUT_MONTHS}: n={len(df_series)}")

        train = df_series.iloc[:-HOLDOUT_MONTHS].copy()
        test = df_series.iloc[-HOLDOUT_MONTHS:].copy()

        # ---------- Prophet ----------
        prophet_res = {"mse": np.nan, "rmse": np.nan, "mae": np.nan, "mape": np.nan}
        prophet_forecast_full = None
        try:
            m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
            m.fit(train[['ds','y']]) # Explicitly selecting ds and y
            # forecast for holdout
            fut = m.make_future_dataframe(periods=HOLDOUT_MONTHS, freq="MS")
            fc = m.predict(fut)
            pred_prop = fc[['ds','yhat']].tail(HOLDOUT_MONTHS)['yhat'].values
            prophet_res = metrics(test['y'].values, pred_prop)

            # fit on full series (df_series already has ds/y)
            m_full = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
            m_full.fit(df_series[['ds','y']]) # Explicitly selecting ds and y
            fut_full = m_full.make_future_dataframe(periods=FORECAST_MONTHS, freq="MS")
            fc_full = m_full.predict(fut_full)
            prophet_forecast_full = fc_full[['ds','yhat','yhat_lower','yhat_upper']].copy()
            prophet_forecast_full['ds'] = pd.to_datetime(prophet_forecast_full['ds'])
            # save prophet forecast tail to csv
            fname = safe_filename(f"{prod}_{st}_prophet_forecast.csv")
            p_save = os.path.join(PROPHET_FORECASTS_FOLDER, fname)
            prophet_forecast_full.to_csv(p_save, index=False)
        except Exception as e:
            print("Prophet failed for pair:", e)
            prophet_res = {"mse": np.nan, "rmse": np.nan, "mae": np.nan, "mape": np.nan}

        # ---------- SARIMA (pmdarima preferred) ----------
        sarima_res = {"mse": np.nan, "rmse": np.nan, "mae": np.nan, "mape": np.nan}
        sarima_forecast_full = None
        try:
            if PM_AVAILABLE:
                arima = pm.auto_arima(train['y'].values, seasonal=True, m=12, error_action='ignore', suppress_warnings=True)
                pred_sarima = arima.predict(n_periods=HOLDOUT_MONTHS)
                sarima_res = metrics(test['y'].values, pred_sarima)
                # fit full and forecast
                arima.fit(df_series['y'].values)
                sarima_forecast_full = arima.predict(n_periods=FORECAST_MONTHS)
                # save sarima forecast full to csv with dates
                dates_full = pd.date_range(start=df_series['ds'].max() + pd.DateOffset(months=1), periods=FORECAST_MONTHS, freq='MS')
                sarima_df = pd.DataFrame({"ds": dates_full, "yhat_sarima": sarima_forecast_full})
                fname = safe_filename(f"{prod}_{st}_sarima_forecast.csv")
                sarima_df.to_csv(os.path.join(SARIMA_FORECASTS_FOLDER, fname), index=False)
            elif SM_AVAILABLE:
                mod = sm.tsa.statespace.SARIMAX(train['y'].values, order=(1,1,1), seasonal_order=(1,1,1,12),
                                                enforce_stationarity=False, enforce_invertibility=False)
                res = mod.fit(disp=False)
                pred_sarima = res.get_forecast(steps=HOLDOUT_MONTHS).predicted_mean
                sarima_res = metrics(test['y'].values, pred_sarima)
                # fit on full and forecast
                modf = sm.tsa.statespace.SARIMAX(df_series['y'].values, order=(1,1,1), seasonal_order=(1,1,1,12),
                                                 enforce_stationarity=False, enforce_invertibility=False)
                resf = modf.fit(disp=False)
                sarima_forecast_full = resf.get_forecast(steps=FORECAST_MONTHS).predicted_mean
                dates_full = pd.date_range(start=df_series['ds'].max() + pd.DateOffset(months=1), periods=FORECAST_MONTHS, freq='MS')
                sarima_df = pd.DataFrame({"ds": dates_full, "yhat_sarima": sarima_forecast_full})
                fname = safe_filename(f"{prod}_{st}_sarima_forecast.csv")
                sarima_df.to_csv(os.path.join(SARIMA_FORECASTS_FOLDER, fname), index=False)
            else:
                raise ImportError("No SARIMA backend available (pmdarima or statsmodels)")
        except Exception as e:
            print("SARIMA failed for pair:", e)
            sarima_res = {"mse": np.nan, "rmse": np.nan, "mae": np.nan, "mape": np.nan}

        # Choose model by RMSE (lower is better). If both NaN -> Prophet by default if available
        chosen = None
        try:
            p_rmse = prophet_res.get("rmse", np.nan)
            s_rmse = sarima_res.get("rmse", np.nan)
            if not np.isnan(p_rmse) and not np.isnan(s_rmse):
                chosen = "PROPHET" if p_rmse <= s_rmse else "SARIMA"
            elif not np.isnan(p_rmse):
                chosen = "PROPHET"
            elif not np.isnan(s_rmse):
                chosen = "SARIMA"
            else:
                chosen = "FAILED"
        except Exception:
            chosen = "FAILED"

        # save a compact metrics row
        row = {
            "Product": prod, "State": st, "n_months": int(n_months),
            "prophet_mse": prophet_res.get("mse"), "prophet_rmse": prophet_res.get("rmse"),
            "prophet_mae": prophet_res.get("mae"), "prophet_mape": prophet_res.get("mape"),
            "sarima_mse": sarima_res.get("mse"), "sarima_rmse": sarima_res.get("rmse"),
            "sarima_mae": sarima_res.get("mae"), "sarima_mape": sarima_res.get("mape"),
            "chosen_model": chosen, "notes": ""
        }
        metrics_master = pd.concat([metrics_master, pd.DataFrame([row])], ignore_index=True)
        metrics_master.to_csv(METRICS_OUT, index=False)

        print(f"Done: {prod} — {st} | chosen: {chosen} | prophet_rmse={prophet_res.get('rmse'):.3f} sarima_rmse={sarima_res.get('rmse') if not pd.isna(sarima_res.get('rmse')) else 'NA'}")

    except Exception as e_main:
        tb = traceback.format_exc()
        print("Fatal error for pair", prod, st, " — logged to failures.")
        failure_row = {"Product": prod, "State": st, "error": str(e_main), "traceback": tb}
        failures = pd.concat([failures, pd.DataFrame([failure_row])], ignore_index=True)
        failures.to_csv(FAILURES_OUT, index=False)
        # continue to next pair
        continue

print("\nBATCH RUN COMPLETE.")
print("Metrics saved to:", METRICS_OUT)
print("Prophet forecasts folder:", PROPHET_FORECASTS_FOLDER)
print("SARIMA forecasts folder:", SARIMA_FORECASTS_FOLDER)
print("Failures (if any):", FAILURES_OUT)


Ensuring prophet and dependencies are installed...
Loading data...
Monthly rows: (21574, 6)
Valid pairs: (447, 3)
Preparing to run 447 pairs (of 447 available)
Skipping CHECKING OR SAVINGS ACCOUNT — AZ (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — CA (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — CT (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — FL (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — GA (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — IL (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — MI (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — NC (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — NJ (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — NV (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — NY (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — OH (already processed)
Skipping CHECKING OR SAVINGS ACCOUNT — PA (already processed)
Skipping CHECKING OR SAVINGS ACCOU

checking which model is good

In [ ]:
import pandas as pd

metrics_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/forecasts_batch/metrics_all_pairs.csv"

df_metrics = pd.read_csv(metrics_path)
print(df_metrics.shape)
df_metrics.head(10)


(447, 13)


,Product,State,n_months,prophet_mse,prophet_rmse,prophet_mae,prophet_mape,sarima_mse,sarima_rmse,sarima_mae,sarima_mape,chosen_model,notes
0,CHECKING OR SAVINGS ACCOUNT,AZ,60,NaN,NaN,NaN,NaN,31.287862,5.593555,4.536419,0.573297,SARIMA,NaN
1,CHECKING OR SAVINGS ACCOUNT,CA,60,NaN,NaN,NaN,NaN,1412.749238,37.586557,31.304172,0.778627,SARIMA,NaN
2,CHECKING OR SAVINGS ACCOUNT,CT,60,NaN,NaN,NaN,NaN,8.313220,2.883265,2.343027,1.259259,SARIMA,NaN
3,CHECKING OR SAVINGS ACCOUNT,FL,60,NaN,NaN,NaN,NaN,752.921871,27.439422,22.880008,0.730775,SARIMA,NaN
4,CHECKING OR SAVINGS ACCOUNT,GA,60,NaN,NaN,NaN,NaN,192.329825,13.868303,10.354274,0.563838,SARIMA,NaN
5,CHECKING OR SAVINGS ACCOUNT,IL,60,NaN,NaN,NaN,NaN,197.359774,14.048479,11.677959,1.040884,SARIMA,NaN
6,CHECKING OR SAVINGS ACCOUNT,MI,60,NaN,NaN,NaN,NaN,36.044868,6.003738,4.569710,0.499813,SARIMA,NaN
7,CHECKING OR SAVINGS ACCOUNT,NC,60,NaN,NaN,NaN,NaN,115.269280,10.736353,7.608383,0.880791,SARIMA,NaN
8,CHECKING OR SAVINGS ACCOUNT,NJ,60,NaN,NaN,NaN,NaN,82.621974,9.089663,7.349154,1.060533,SARIMA,NaN
9,CHECKING OR SAVINGS ACCOUNT,NV,60,NaN,NaN,NaN,NaN,221.315798,14.876686,14.595273,1.818962,SARIMA,NaN


In [14]:
import pandas as pd

df = pd.read_csv(
    "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/forecasts_batch/sarima_forecasts/CHECKING_OR_SAVINGS_ACCOUNT_CA_sarima_forecast.csv"
)

df
pair_df.tail(12)


,Product,State,year,month_num,complaint_count,date
12,STUDENT LOAN,OK,2024,4,2,2024-04-01
13,STUDENT LOAN,OK,2024,6,2,2024-06-01
14,STUDENT LOAN,OK,2024,7,2,2024-07-01
15,STUDENT LOAN,OK,2024,9,1,2024-09-01
16,STUDENT LOAN,OK,2024,10,1,2024-10-01
17,STUDENT LOAN,OK,2024,12,2,2024-12-01
18,STUDENT LOAN,OK,2025,2,1,2025-02-01
19,STUDENT LOAN,OK,2025,3,3,2025-03-01
20,STUDENT LOAN,OK,2025,4,1,2025-04-01
21,STUDENT LOAN,OK,2025,5,1,2025-05-01


In [19]:
import os
import pandas as pd
import numpy as np

# === PATHS ===
metrics_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/forecasts_batch/metrics_all_pairs.csv"
sarima_folder = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/forecasts_batch/sarima_forecasts/"

# === LOAD METRICS ===
metrics = pd.read_csv(metrics_path)

print("Metrics loaded:", metrics.shape)

# === STABILITY RULES (STRICT) ===
def is_stable(row):
    """
    Returns True if SARIMA forecast for a pair is considered reliable.
    """
    # 1. SARIMA must have valid metrics
    if pd.isna(row["sarima_rmse"]) or pd.isna(row["sarima_mae"]) or pd.isna(row["sarima_mape"]):
        return False

    # 2. Hard thresholds
    if row["sarima_rmse"] > 30:   # too high variance
        return False
    if row["sarima_mae"] > 20:    # too high error
        return False
    if row["sarima_mape"] > 1:    # > 100% error
        return False

    return True


# === APPLY RULES TO METRICS ===
metrics["stable_by_metrics"] = metrics.apply(is_stable, axis=1)

print("Stable by metric thresholds:", metrics["stable_by_metrics"].sum())

# === CHECK FORECAST SHAPE (detect flat line or weird shape) ===
def forecast_is_flat(filepath):
    """Detects constant-value (flat) forecasts."""
    try:
        df = pd.read_csv(filepath)
        y = df.iloc[:, 1].values  # second column = forecast values
        return np.allclose(y, y[0])  # all values same
    except:
        return True  # if file cannot open => treat as unstable


def forecast_is_unstable_jump(filepath):
    """Detects explosive patterns (jump > 300% or negative values)."""
    try:
        df = pd.read_csv(filepath)
        y = df.iloc[:, 1].values

        if np.any(y < 0):
            return True  # negatives = unstable

        # large jumps relative to previous value
        for i in range(1, len(y)):
            if y[i-1] != 0 and abs(y[i] - y[i-1]) / abs(y[i-1]) > 3.0:
                return True

        return False
    except:
        return True


# === COMBINE RULES ===
stable_pairs = []

for idx, row in metrics.iterrows():
    prod = row["Product"]
    state = row["State"]
    fname = f"{prod.replace(' ', '_')}_{state}_sarima_forecast.csv"
    fpath = os.path.join(sarima_folder, fname)

    # skip if file missing
    if not os.path.exists(fpath):
        continue

    # rule 1: metrics must be stable
    if not row["stable_by_metrics"]:
        continue

    # rule 2: forecast must not be flat
    if forecast_is_flat(fpath):
        continue

    # rule 3: forecast must not jump in insane way
    if forecast_is_unstable_jump(fpath):
        continue

    # Passed all checks
    stable_pairs.append((prod, state))


# === PRINT RESULTS ===
print("\n================ RESULTS ================")
print(f"Total pairs in metrics: {metrics.shape[0]}")
print(f"Stable pairs (based on metrics + forecast shape): {len(stable_pairs)}")

print("\nSample stable pairs:")
print(stable_pairs[:10])

# DO NOT SAVE ANYTHING YET (as requested)


Metrics loaded: (447, 13)
Stable by metric thresholds: 217

================ RESULTS ================
Total pairs in metrics: 447
Stable pairs (based on metrics + forecast shape): 111

Sample stable pairs:
[('CHECKING OR SAVINGS ACCOUNT', 'GA'), ('CHECKING OR SAVINGS ACCOUNT', 'MI'), ('CHECKING OR SAVINGS ACCOUNT', 'NC'), ('CHECKING OR SAVINGS ACCOUNT', 'NY'), ('CHECKING OR SAVINGS ACCOUNT', 'TX'), ('DEBT COLLECTION', 'NJ'), ('DEBT COLLECTION', 'TN'), ('DEBT COLLECTION', 'VA'), ('MORTGAGE', 'CA'), ('MORTGAGE', 'FL')]


checkoint

In [21]:
import pandas as pd

# your stable pairs list
stable_pairs_list = stable_pairs   # whatever variable holds them

# convert list → dataframe
stable_pairs_df = pd.DataFrame(stable_pairs_list, columns=["Product", "State"])

# save
out_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/stable_pairs.csv"
stable_pairs_df.to_csv(out_path, index=False)

print("Saved:", out_path)
stable_pairs_df.head()


Saved: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/stable_pairs.csv


,Product,State
0,CHECKING OR SAVINGS ACCOUNT,GA
1,CHECKING OR SAVINGS ACCOUNT,MI
2,CHECKING OR SAVINGS ACCOUNT,NC
3,CHECKING OR SAVINGS ACCOUNT,NY
4,CHECKING OR SAVINGS ACCOUNT,TX


In [22]:
import os

path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/forecasts_batch/sarima_forecasts"
print(len(os.listdir(path)))
print(os.listdir(path)[:20])


364
['CHECKING_OR_SAVINGS_ACCOUNT_AZ_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_CA_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_CT_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_FL_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_GA_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_IL_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_MI_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_NC_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_NJ_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_NV_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_NY_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_OH_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_PA_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_TX_sarima_forecast.csv', 'CHECKING_OR_SAVINGS_ACCOUNT_VA_sarima_forecast.csv', 'DEBT_COLLECTION_AL_sarima_forecast.csv', 'DEBT_COLLECTION_AZ_sarima_forecast.csv', 'DEBT_COLLECTION_CA_sarima_forecast.csv', 'DEBT_COLLECTION_CO_sarima_forecast.csv', 'DEBT_COLLECTION_

In [23]:
import pandas as pd
import os

# === Paths ===
stable_pairs_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/stable_pairs.csv"
sarima_folder = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/forecasts_batch/sarima_forecasts"
output_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/final_forecast_stable.csv"

# === Load stable pairs ===
stable_df = pd.read_csv(stable_pairs_path)

records = []

# Loop through each stable pair and load its SARIMA forecast
for _, row in stable_df.iterrows():
    prod = row['Product']
    state = row['State']

    # Build the filename used in your SARIMA folder
    file_name = f"{prod.replace(' ', '_')}_{state}_sarima_forecast.csv"

    file_path = os.path.join(sarima_folder, file_name)

    if not os.path.exists(file_path):
        print("Missing forecast file for:", prod, state)
        continue

    fc = pd.read_csv(file_path)

    # Add Product, State, Model
    fc['Product'] = prod
    fc['State'] = state
    fc['model'] = "SARIMA"

    records.append(fc)

# Combine all
if records:
    final_df = pd.concat(records, ignore_index=True)
    final_df.to_csv(output_path, index=False)
    print("Saved:", output_path)
    print("Shape:", final_df.shape)
else:
    print("No forecasts loaded!")


Saved: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/final_forecast_stable.csv
Shape: (1332, 5)


In [ ]:
import pandas as pd

# Paths
forecast_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/final_forecast_stable.csv"
root_summary_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_summary.csv"
root_rows_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows.csv"

# Load
df_forecast = pd.read_csv(forecast_path, parse_dates=["ds"])
df_summary = pd.read_csv(root_summary_path)
df_rows = pd.read_csv(root_rows_path)

# Show shapes
print("Forecast shape:", df_forecast.shape)
print("Root summary shape:", df_summary.shape)
print("Root rows shape:", df_rows.shape)

# Column types
print("\n=== FORECAST DTYPES ===")
print(df_forecast.dtypes)

print("\n=== SUMMARY DTYPES ===")
print(df_summary.dtypes)

print("\n=== ROWS DTYPES ===")
print(df_rows.dtypes)

# Null checks
print("\n=== NULL COUNTS (FORECAST) ===")
print(df_forecast.isnull().sum())

print("\n=== NULL COUNTS (SUMMARY) ===")
print(df_summary.isnull().sum())

print("\n=== NULL COUNTS (ROWS) ===")
print(df_rows.isnull().sum())

# Quick preview
print("\n=== FORECAST HEAD ===")
print(df_forecast.head())

print("\n=== SUMMARY HEAD ===")
print(df_summary.head())

print("\n=== ROWS HEAD ===")
print(df_rows.head())


In [31]:
import pandas as pd

monthly_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated.csv"

# Load
m = pd.read_csv(monthly_path)

# Valid states list
valid_states = [
'AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI',
'MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT',
'VT','VA','WA','WV','WI','WY','DC','PR','VI','GU','AS','MP'
]

print("Before cleaning:", m['State'].nunique())

# Keep only valid state rows
m_clean = m[m['State'].isin(valid_states)].copy()

print("After cleaning:", m_clean['State'].nunique())
print("Rows removed:", len(m) - len(m_clean))

# Save cleaned version
cleaned_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated_clean.csv"
m_clean.to_csv(cleaned_path, index=False)

print("\nCleaned file saved to:", cleaned_path)

Before cleaning: 66
After cleaning: 56
Rows removed: 124

Cleaned file saved to: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated_clean.csv


In [ ]:
import pandas as pd

summary_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_summary.csv"
rows_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows.csv"

# Load
summ = pd.read_csv(summary_path)
rows = pd.read_csv(rows_path)

print("=== SUMMARY CHECK ===")
print("Shape:", summ.shape)
print("Unique Products:", summ['Product'].unique())
print("Nulls:")
print(summ.isnull().sum())
print("\nSample:")
print(summ.head())

print("\n=== ROWS CHECK ===")
print("Shape:", rows.shape)
print("Unique Products:", rows['Product'].unique()[:20])
print("Unique States (first 20):", rows['State'].unique()[:20])
print("Null count:")
print(rows.isnull().sum())
print("\nSample:")
print(rows.head(5))

In [38]:
import pandas as pd

# Load file
path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows.csv"
df = pd.read_csv(path)

# Mapping: Full state name → Abbreviation
state_map = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN',
    'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI',
    'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Convert using mapping
df['State'] = df['State'].map(state_map)

# Save cleaned file
save_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows_clean.csv"
df.to_csv(save_path, index=False)

print("Saved cleaned root cause rows at:", save_path)
print("Unique states:", df['State'].unique())
print("Null states:", df['State'].isna().sum())
print("Shape:", df.shape)

Saved cleaned root cause rows at: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows_clean.csv
Unique states: ['NY' 'MI' 'CA' 'MN' 'FL' 'MS' 'VA' 'NJ' 'KS' 'OR' 'AZ' 'MA' 'OH' 'UT'
 'IL' 'CT' 'MT' 'TN' 'PA' 'TX' 'GA' 'IN' 'SC' 'KY' 'NV' 'LA' 'OK' 'CO'
 'AL' 'RI' 'DE' 'WA' 'MD' 'NM' 'AR' 'MO' 'NC' 'AK' 'ME' 'WI' nan 'WV' 'NE'
 'SD' 'HI' 'NH' 'IA' 'WY' 'ND' 'VT' 'ID']
Null states: 224
Shape: (40000, 23)


In [39]:
df_clean = df.dropna(subset=['State']).copy()

save_path2 = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows_clean_final.csv"
df_clean.to_csv(save_path2, index=False)

print("Final cleaned root cause file saved:", save_path2)
print("Final shape:", df_clean.shape)
print("Remaining null states:", df_clean['State'].isna().sum())

Final cleaned root cause file saved: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows_clean_final.csv
Final shape: (39776, 23)
Remaining null states: 0


validating files

In [40]:
df = pd.read_csv("/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated_clean.csv")
print(df.shape)
print(df.State.unique())
print(df.isnull().sum())
print(df.head())

(21450, 6)
['AK' 'AL' 'AR' 'AS' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'GU' 'HI'
 'IA' 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS'
 'MT' 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'PR'
 'RI' 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VI' 'VT' 'WA' 'WI' 'WV' 'WY' 'MP']
Product            0
State              0
year               0
month_num          0
complaint_count    0
date               0
dtype: int64
                       Product State  year  month_num  complaint_count  \
0  CHECKING OR SAVINGS ACCOUNT    AK  2021          2                1   
1  CHECKING OR SAVINGS ACCOUNT    AK  2021          7                2   
2  CHECKING OR SAVINGS ACCOUNT    AK  2022          6                1   
3  CHECKING OR SAVINGS ACCOUNT    AK  2022         10                1   
4  CHECKING OR SAVINGS ACCOUNT    AK  2022         11                1   

         date  
0  2021-02-01  
1  2021-07-01  
2  2022-06-01  
3  2022-10-01  
4  2022-11-01  


In [41]:
f = pd.read_csv("/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/final_forecast_stable.csv")
print("Shape:", f.shape)
print("Unique States:", sorted(f.State.unique()))
print("Nulls:", f.isnull().sum())
print("Head:\n", f.head())

Shape: (1332, 5)
Unique States: ['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'WA', 'WI', 'WV', 'WY']
Nulls: ds             0
yhat_sarima    0
Product        0
State          0
model          0
dtype: int64
Head:
            ds  yhat_sarima                      Product State   model
0  2025-11-01    33.479601  CHECKING OR SAVINGS ACCOUNT    GA  SARIMA
1  2025-12-01    29.904047  CHECKING OR SAVINGS ACCOUNT    GA  SARIMA
2  2026-01-01    26.146556  CHECKING OR SAVINGS ACCOUNT    GA  SARIMA
3  2026-02-01    28.856418  CHECKING OR SAVINGS ACCOUNT    GA  SARIMA
4  2026-03-01    29.480944  CHECKING OR SAVINGS ACCOUNT    GA  SARIMA


In [42]:
rc = pd.read_csv("/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_summary.csv")
rc_rows = pd.read_csv("/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows_clean_final.csv")

print("SUMMARY SHAPE:", rc.shape)
print("ROWS SHAPE:", rc_rows.shape)

print("\nSUMMARY PRODUCTS:", sorted(rc.Product.unique()))
print("ROWS PRODUCTS:", sorted(rc_rows.Product.unique()))

print("\nSUMMARY NULLS:\n", rc.isnull().sum())
print("\nROWS NULLS:\n", rc_rows.isnull().sum())

print("\nSAMPLE SUMMARY:\n", rc.head())
print("\nSAMPLE ROWS:\n", rc_rows.head())

SUMMARY SHAPE: (28, 5)
ROWS SHAPE: (39776, 23)

SUMMARY PRODUCTS: ['CHECKING OR SAVINGS ACCOUNT', 'CREDIT CARD', 'CREDIT CARD OR PREPAID CARD', 'CREDIT REPORTING OR OTHER PERSONAL CONSUMER REPORTS', 'CREDIT REPORTING, CREDIT REPAIR SERVICES, OR OTHER PERSONAL CONSUMER REPORTS', 'DEBT COLLECTION', 'DEBT OR CREDIT MANAGEMENT', 'MONEY TRANSFER, VIRTUAL CURRENCY, OR MONEY SERVICE', 'MORTGAGE', 'PAYDAY LOAN, TITLE LOAN, OR PERSONAL LOAN', 'PAYDAY LOAN, TITLE LOAN, PERSONAL LOAN, OR ADVANCE LOAN', 'PREPAID CARD', 'STUDENT LOAN', 'VEHICLE LOAN OR LEASE']
ROWS PRODUCTS: ['CHECKING OR SAVINGS ACCOUNT', 'CREDIT CARD', 'CREDIT CARD OR PREPAID CARD', 'CREDIT REPORTING OR OTHER PERSONAL CONSUMER REPORTS', 'CREDIT REPORTING, CREDIT REPAIR SERVICES, OR OTHER PERSONAL CONSUMER REPORTS', 'DEBT COLLECTION', 'DEBT OR CREDIT MANAGEMENT', 'MONEY TRANSFER, VIRTUAL CURRENCY, OR MONEY SERVICE', 'MORTGAGE', 'PAYDAY LOAN, TITLE LOAN, OR PERSONAL LOAN', 'PAYDAY LOAN, TITLE LOAN, PERSONAL LOAN, OR ADVANCE LOAN', 

In [45]:
# Rebuild clean BI master: historical rows get topics; forecasts remain topic-free.
# Paste and run in Colab (paths from your notebook).
import pandas as pd
import os

# --- PATHS (change only if your files are elsewhere) ---
monthly_clean_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/monthly_aggregated_clean.csv"
root_summary_path  = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_summary.csv"
root_rows_clean_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/root_cause_rows_clean_final.csv"
forecast_path      = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/final_forecast_stable.csv"
out_master_path    = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_DATA.csv"

# --- LOAD ---
print("Loading files...")
monthly = pd.read_csv(monthly_clean_path, parse_dates=["date"])
summary = pd.read_csv(root_summary_path)   # product-level topic summary (no state)
forecast = pd.read_csv(forecast_path, parse_dates=["ds"])
rows = pd.read_csv(root_rows_clean_path, parse_dates=["date_received"])

# --- NORMALIZE / RENAME ---
# monthly: date -> ds, complaint_count -> actual_count
monthly = monthly.rename(columns={"date":"ds","complaint_count":"actual_count","Product":"Product","State":"State"})
monthly = monthly[["ds","Product","State","actual_count","year","month_num"]]

# summary: expect columns Product, topic, count, keywords, interpretation
expected_summary_cols = ["Product","topic","count","keywords","interpretation"]
if not set(expected_summary_cols).issubset(set(summary.columns)):
    raise RuntimeError(f"root_cause_summary missing expected cols: {expected_summary_cols}")

# forecast: ensure single forecast_count column
# common names observed: yhat_sarima or yhat; convert to forecast_count
if "yhat_sarima" in forecast.columns:
    forecast = forecast.rename(columns={"yhat_sarima":"forecast_count"})
elif "yhat" in forecast.columns:
    forecast = forecast.rename(columns={"yhat":"forecast_count"})
else:
    # attempt to detect numeric forecast column
    numeric_cols = forecast.select_dtypes(include="number").columns.tolist()
    # remove any index-like 'Unnamed' columns
    numeric_cols = [c for c in numeric_cols if not str(c).lower().startswith("unnamed")]
    if len(numeric_cols) == 1:
        forecast = forecast.rename(columns={numeric_cols[0]:"forecast_count"})
    else:
        raise RuntimeError("Can't determine forecast column in forecast file. Rename forecast numeric col to yhat_sarima or yhat.")

# ensure forecast has ds,Product,State,forecast_count,model
required_fc_cols = ["ds","Product","State","forecast_count"]
if not set(required_fc_cols).issubset(set(forecast.columns)):
    raise RuntimeError(f"Forecast file missing required columns: {required_fc_cols}. Found: {list(forecast.columns)}")

# keep only needed forecast cols + model if present
fc_cols = ["ds","Product","State","forecast_count"]
if "model" in forecast.columns:
    fc_cols.append("model")
forecast = forecast[fc_cols].copy()

# --- BUILD historical_with_topics (attach summary to every monthly (Product,State,ds) row) ---
# Important: this intentionally expands monthly rows by topic counts (correct for historical analysis).
hist = monthly.copy()
# merge (left) hist x summary by Product -> topic-level historical rows
hist_topics = hist.merge(summary, how="left", on="Product", suffixes=("","_summary"))

# For clarity in BI, rename summary.count -> topic_count
hist_topics = hist_topics.rename(columns={"count":"topic_count","keywords":"topic_keywords","interpretation":"topic_interpretation"})

# Reorder columns for historical
hist_cols = ["ds","Product","State","actual_count","topic","topic_count","topic_keywords","topic_interpretation","year","month_num"]
hist_topics = hist_topics[hist_cols]

# --- Prepare forecast-only table (no topic columns) ---
# Add topic columns as NaN to match schema but keep them empty
forecast_out = forecast.copy()
forecast_out["topic"] = pd.NA
forecast_out["topic_count"] = pd.NA
forecast_out["topic_keywords"] = pd.NA
forecast_out["topic_interpretation"] = pd.NA
# forecast doesn't have actual_count/year/month_num (we keep them NaN)
forecast_out["actual_count"] = pd.NA
forecast_out["year"] = forecast_out["ds"].dt.year
forecast_out["month_num"] = forecast_out["ds"].dt.month

# reorder forecast columns to match hist_topics
forecast_out = forecast_out[["ds","Product","State","actual_count","topic","topic_count","topic_keywords","topic_interpretation","year","month_num","forecast_count"] + (["model"] if "model" in forecast_out.columns else [])]

# --- CONCATENATE master (historical with topics) + forecasts (topic-free) ---
# We'll keep forecast_count column in master; historical rows will have forecast_count = NaN
hist_topics["forecast_count"] = pd.NA
if "model" not in hist_topics.columns:
    hist_topics["model"] = pd.NA

# align columns exactly
master_cols = ["ds","Product","State","actual_count","topic","topic_count","topic_keywords","topic_interpretation","year","month_num","forecast_count","model"]
hist_topics = hist_topics[master_cols]
# ensure forecast_out contains the same columns (it does)
forecast_out = forecast_out[master_cols]

# final concat
master = pd.concat([hist_topics, forecast_out], ignore_index=True, sort=False)

# --- DEDUP / SANITY ---
# Remove exact duplicate rows
before = len(master)
master = master.drop_duplicates()
after = len(master)

# Ensure no rows have both actual_count and forecast_count non-null (that would be bad)
both_nonnull = master[master["actual_count"].notna() & master["forecast_count"].notna()]
if len(both_nonnull) > 0:
    print("WARNING: rows where both actual_count AND forecast_count present (should be 0). Showing first 5:")
    display(both_nonnull.head(5))

# --- Save ---
os.makedirs(os.path.dirname(out_master_path), exist_ok=True)
master.to_csv(out_master_path, index=False)
print(f"\nFINAL BI MASTER SAVED AT: {out_master_path}")
print(f"FINAL SHAPE: {master.shape}")
print(f"Removed exact duplicates during build: {before - after}")

# --- DIAGNOSTICS (paste this output back to me) ---
print("\n--- DIAGNOSTICS ---")
print("Date range: ", master['ds'].min(), "to", master['ds'].max())
print("Unique Products:", master['Product'].nunique())
print("Unique States:", sorted(master['State'].dropna().unique()))
print("Total rows:", len(master))
print("Historical rows (actual_count non-null):", master['actual_count'].notna().sum())
print("Forecast rows (forecast_count non-null):", master['forecast_count'].notna().sum())
print("Rows with topic info (topic non-null):", master['topic'].notna().sum())
print("Rows with both actual & forecast non-null (should be 0):", len(master[master['actual_count'].notna() & master['forecast_count'].notna()]))
print("\nNull counts (top columns):")
print(master[['actual_count','forecast_count','topic','topic_count','topic_keywords','topic_interpretation','model']].isna().sum())

# show small samples
print("\nSAMPLE HISTORICAL (head):")
display(master[master['actual_count'].notna()].head(5))
print("\nSAMPLE FORECAST (head):")
display(master[master['forecast_count'].notna()].head(5))

Loading files...

FINAL BI MASTER SAVED AT: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_DATA.csv
FINAL SHAPE: (44232, 12)
Removed exact duplicates during build: 0

--- DIAGNOSTICS ---
Date range:  2020-11-01 00:00:00 to 2026-10-01 00:00:00
Unique Products: 14
Unique States: ['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']
Total rows: 44232
Historical rows (actual_count non-null): 42900
Forecast rows (forecast_count non-null): 1332
Rows with topic info (topic non-null): 42900
Rows with both actual & forecast non-null (should be 0): 0

Null counts (top columns):
actual_count             1332
forecast_count          42900
topic                    1332
to

,ds,Product,State,actual_count,topic,topic_count,topic_keywords,topic_interpretation,year,month_num,forecast_count,model
0,2021-02-01,CHECKING OR SAVINGS ACCOUNT,AK,1,1,844,"bank, was, card, they, the",Card-related dispute or transaction issue,2021,2,NaN,NaN
1,2021-02-01,CHECKING OR SAVINGS ACCOUNT,AK,1,22,175,"wells, fargo, was, they, money",General account or transaction dispute,2021,2,NaN,NaN
2,2021-07-01,CHECKING OR SAVINGS ACCOUNT,AK,2,1,844,"bank, was, card, they, the",Card-related dispute or transaction issue,2021,7,NaN,NaN
3,2021-07-01,CHECKING OR SAVINGS ACCOUNT,AK,2,22,175,"wells, fargo, was, they, money",General account or transaction dispute,2021,7,NaN,NaN
4,2022-06-01,CHECKING OR SAVINGS ACCOUNT,AK,1,1,844,"bank, was, card, they, the",Card-related dispute or transaction issue,2022,6,NaN,NaN



SAMPLE FORECAST (head):


,ds,Product,State,actual_count,topic,topic_count,topic_keywords,topic_interpretation,year,month_num,forecast_count,model
42900,2025-11-01,CHECKING OR SAVINGS ACCOUNT,GA,<NA>,<NA>,<NA>,NaN,NaN,2025,11,33.479601,SARIMA
42901,2025-12-01,CHECKING OR SAVINGS ACCOUNT,GA,<NA>,<NA>,<NA>,NaN,NaN,2025,12,29.904047,SARIMA
42902,2026-01-01,CHECKING OR SAVINGS ACCOUNT,GA,<NA>,<NA>,<NA>,NaN,NaN,2026,1,26.146556,SARIMA
42903,2026-02-01,CHECKING OR SAVINGS ACCOUNT,GA,<NA>,<NA>,<NA>,NaN,NaN,2026,2,28.856418,SARIMA
42904,2026-03-01,CHECKING OR SAVINGS ACCOUNT,GA,<NA>,<NA>,<NA>,NaN,NaN,2026,3,29.480944,SARIMA


In [44]:
import pandas as pd

bi = pd.read_csv("/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_DATA.csv")

print("Rows:", bi.shape[0])
print("\nUnique States:", sorted(bi["State"].unique()))
print("\nCount per State:")
print(bi["State"].value_counts())

print("\nAny null states:", bi["State"].isna().sum())

print("\nRows where forecast_count is null but model == 'SARIMA':")
print(bi[(bi["model"]=="SARIMA") & (bi["forecast_count"].isna())].head())

print("\nRows where actual_count is null during historical period:")
hist_only = bi[bi["ds"] < "2025-11-01"]
print(hist_only[hist_only["actual_count"].isna()].head())

Rows: 45564

Unique States: ['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']

Count per State:
State
CA    1274
NY    1238
NC    1228
TX    1226
GA    1220
FL    1216
MI    1202
VA    1180
AZ    1174
OH    1158
IL    1150
TN    1146
SC    1140
NJ    1138
PA    1118
MA    1096
IN    1088
CO    1080
MD    1076
MO    1056
AL    1026
WA    1018
LA    1010
NV     944
WI     944
CT     932
MN     932
KY     904
OR     898
MS     884
UT     882
KS     862
OK     854
AR     822
DE     792
NM     752
DC     752
IA     698
ID     670
WV     644
NE     590
NH     564
RI     550
HI     482
ME     476
MT     426
PR     378
SD     370
ND     334
AK     316
WY     280
VT     238
VI      76
GU      50
MP       6
AS    

In [46]:
import pandas as pd

path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_DATA.csv"
df = pd.read_csv(path, dtype=str)  # load as string so we see corruption

# Show unique problematic values
bad = df['forecast_count'].unique()
print("Unique forecast_count values:")
for v in bad:
    print(repr(v))

Unique forecast_count values:
nan
'33.47960087839368'
'29.90404689119805'
'26.146556228203675'
'28.85641786901797'
'29.480943647608225'
'29.07084926808607'
'29.55756034599456'
'30.02396639789429'
'30.258198872049835'
'30.574441639606725'
'30.925468158932137'
'31.243174109614174'
'13.23573958820009'
'12.35329548600338'
'11.230382635477856'
'12.511684686809083'
'11.70996026340501'
'10.917595677171926'
'10.932572005446833'
'11.625107919311976'
'12.631781183494317'
'11.557824052441266'
'9.536637675755443'
'8.911510067006628'
'9.898023389757393'
'8.105756892482818'
'9.117192386851183'
'8.690864180845509'
'9.031512023088826'
'8.962239599063926'
'9.11109949241894'
'9.144001250710952'
'9.238566221947412'
'9.300334583887311'
'9.379546253918493'
'9.449480678551016'
'18.772035848184544'
'17.597670114887528'
'17.095780066285975'
'17.548591130865344'
'17.73266255539975'
'17.823295987924162'
'17.979193747097387'
'18.13306615182288'
'18.277825347656737'
'18.424971625834324'
'18.57293572641068'
'18.72

In [49]:
import pandas as pd
import numpy as np

# Load BI master
path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_DATA.csv"
df = pd.read_csv(path)

# Convert ds to datetime
df['ds'] = pd.to_datetime(df['ds'])

# Convert actual_count to numeric (keep NaN)
df['actual_count'] = pd.to_numeric(df['actual_count'], errors='coerce')

# Convert forecast_count to numeric (keep NaN)
df['forecast_count'] = pd.to_numeric(df['forecast_count'], errors='coerce')

# Convert topic_count to numeric (if exists)
if 'count' in df.columns:
    df['topic_count'] = pd.to_numeric(df['count'], errors='coerce')
    df = df.drop(columns=['count'])

# Ensure year and month_num numeric (forecast rows will stay NaN)
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['month_num'] = pd.to_numeric(df['month_num'], errors='coerce')

# Save clean version
clean_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_DATA_CLEAN.csv"
df.to_csv(clean_path, index=False)

print("CLEAN FILE SAVED TO:", clean_path)
print("\nDTYPES:\n", df.dtypes)
print("\nNULL CHECK:\n", df[['actual_count','forecast_count','year','month_num']].isna().sum())

CLEAN FILE SAVED TO: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_DATA_CLEAN.csv

DTYPES:
 ds                      datetime64[ns]
Product                         object
State                           object
actual_count                   float64
topic                          float64
topic_count                    float64
topic_keywords                  object
topic_interpretation            object
year                             int64
month_num                        int64
forecast_count                 float64
model                           object
dtype: object

NULL CHECK:
 actual_count       1332
forecast_count    42900
year                  0
month_num             0
dtype: int64


In [50]:
import pandas as pd

# === LOAD FILE ===
df = pd.read_csv("/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_DATA_CLEAN.csv")

# Ensure ds is datetime
df['ds'] = pd.to_datetime(df['ds'])

# =====================
# 1. BUILD CLEAN MONTHLY HISTORICAL
# =====================

hist = df[df['actual_count'].notna()]

hist_monthly = (
    hist.groupby(['ds', 'Product', 'State'], as_index=False)
        .agg({'actual_count': 'sum'})
)

# =====================
# 2. BUILD CLEAN MONTHLY FORECAST
# =====================

forecast = df[df['forecast_count'].notna()][['ds', 'Product', 'State', 'forecast_count']]
forecast['forecast_count'] = pd.to_numeric(forecast['forecast_count'], errors='coerce')

# =====================
# 3. MERGE THEM TOGETHER
# =====================

final = pd.merge(
    hist_monthly,
    forecast,
    on=['ds', 'Product', 'State'],
    how='outer'
)

# =====================
# 4. SORT & CLEAN
# =====================

final = final.sort_values(['ds', 'Product', 'State'])
final = final.drop_duplicates()

# Create helper columns
final['year'] = final['ds'].dt.year
final['month_num'] = final['ds'].dt.month

# =====================
# 5. SAVE FINAL BI FILE
# =====================

save_path = "/content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_FINAL.csv"
final.to_csv(save_path, index=False)

print("✔ FINAL BI FILE SAVED:", save_path)
print("Shape:", final.shape)
print(final.head())

✔ FINAL BI FILE SAVED: /content/drive/MyDrive/Data_Science_Projects/Customer_Complaint_Trend_Forecasting/data/BI_MASTER_FINAL.csv
Shape: (22782, 7)
          ds                      Product State  actual_count  forecast_count  \
0 2020-11-01  CHECKING OR SAVINGS ACCOUNT    AL           4.0             NaN   
1 2020-11-01  CHECKING OR SAVINGS ACCOUNT    AR           4.0             NaN   
2 2020-11-01  CHECKING OR SAVINGS ACCOUNT    AZ           6.0             NaN   
3 2020-11-01  CHECKING OR SAVINGS ACCOUNT    CA          60.0             NaN   
4 2020-11-01  CHECKING OR SAVINGS ACCOUNT    CO           2.0             NaN   

   year  month_num  
0  2020         11  
1  2020         11  
2  2020         11  
3  2020         11  
4  2020         11  
